In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import os
import optuna
import logging
import xgboost as xgb
import sys
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

"""## Variables"""

current_storage = os.path.join("d:", "data")
vn30f_storage = os.path.join(current_storage, 'vn30f')

start_time = 0
now_time = 9999999999
symbol = 'VN30F1M'
rolling_window = 1 # Số phút muốn dự đoán tiếp theo

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "btc1-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

"""## Formulas"""

def choose_position(roi, trade_threshold = 0.0001):
    pos =0
    # Predict position base on change in future
    if roi > trade_threshold:
        pos = 1
    elif roi < -trade_threshold:
        pos = -1
    else:
        pos = 0

    return pos

def backtest_position_ps(position, price, percentage, periods):
    # Shift positions to align with future price changes and handle NaN by filling with 0
    pos = pd.Series(position, index=pd.Series(price).index).shift(1).fillna(0)
    pos = pd.Series(pos).rolling(periods).sum() #pos for 10 hour predict

    price_array = pd.Series(price).shift(1).fillna(0)

    pos_diff = pos.diff()
    fee = pos_diff*price_array*0.05*0.01

    # Calculate price changes over the given periods
    ch = pd.Series(price) - price_array

    # Calculate total PnL
    total_pnl = pos*ch - fee
    return total_pnl

def calculate_sharpe_ratio(pnl):
    pnl = np.diff(pnl)
    std = np.std(pnl) if np.std(pnl) != 0 else 0.001
    sharpe = np.mean(pnl)/std*np.sqrt(252)
    return sharpe

def sharpe_for_vn30f(y_pred, y_price, trade_threshold, fee_perc, periods):

    # Predict position base on change in future
    pos = [choose_position(roi, trade_threshold) for roi in y_pred]
    pos = np.array(pos)

    # Calculate PNL
    pnl = backtest_position_ps(pos, y_price, fee_perc, periods)
    pnl = np.cumsum(pnl)

    # Standardalize PNL to date
    daily_pnl = [pnl.iloc[i] for i in range(0, len(pnl), 241)]
    daily_pnl = pd.Series(daily_pnl).fillna(0)

    # Calculate Sharpe
    sharpe = calculate_sharpe_ratio(daily_pnl)

    return pos, pnl, daily_pnl, sharpe

def calculate_hitrate(pos_predict, pos_true):
    if len(pos_predict) != len(pos_true):
        raise ValueError("Độ dài của hai mảng không khớp")

    # Tính số lượng dự đoán đúng (các phần tử tương ứng giống nhau)
    correct_predictions = np.sum(pos_predict == pos_true)

    # Tính tỷ lệ hit rate
    hit_rate_value = correct_predictions / len(pos_predict)

    return hit_rate_value

"""# Function for data"""

def scale_data(data):
    scaler = StandardScaler()
    data = np.where(np.isinf(data), np.nan, data)
    data = pd.DataFrame(data)
    data = data.fillna(0)
    scaler.fit(data)
    data=pd.DataFrame(scaler.transform(data), index=data.index, columns=data.columns)

    return data

def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

def split_optuna_data(data):
    """
    Hàm này chia dữ liệu thành các tập train và test để sử dụng trong quá trình tối ưu hóa bằng Optuna.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu train (đã được chuẩn hóa).
    pandas.DataFrame: DataFrame chứa dữ liệu test (đã được chuẩn hóa).
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu train.
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu test.
    """
    # Chia dữ liệu thành tập train và tập hold out
    train_data, _ = split_data(data)

    # Loại bỏ các cột không cần thiết
    if 'Open' in train_data.columns:
        optuna_data = train_data.drop(['Open','High','Low','Close','Volume', 'Return', 'Date', 'time'], axis=1)
    else:
        optuna_data = train_data.drop(['Close', 'Return', 'Date', 'time'], axis=1)

    # Chuẩn hóa dữ liệu
    optuna_data = scale_data(optuna_data)

    # Chia dữ liệu thành tập train và tập test
    X_train, X_valid, y_train, y_valid = train_test_split(optuna_data, train_data['Return'], test_size=0.5, shuffle=False)

    return X_train, X_valid, y_train, y_valid, train_data

data = pd.read_csv('/kaggle/input/new-data/save_data.csv')
data = data[data.Date >= '2021']
def objective(trial, X_train, X_valid, y_train, y_valid, y_price):

    # Select features based on Optuna's suggestions
    selected_features = []

    at_least_one_feature = False

    for col in X_train.columns:
        select_feature = trial.suggest_categorical(col, [0, 1])
        if select_feature:
            selected_features.append(col)
            at_least_one_feature = True

    # If no feature was selected, force selection of at least one feature
    if not at_least_one_feature:
        # Randomly select one feature to be included
        forced_feature = trial.suggest_categorical('forced_feature', X_train.columns.tolist())
        selected_features.append(forced_feature)

    for t in trial.study.trials:
        if t.state != optuna.trial.TrialState.COMPLETE:
            continue
        if t.params == trial.params:
            return np.nan # t.values  # Return the previous value without re-evaluating i

    trade_threshold  = 0.0001

    # Use only the selected features in training
    X_train_selected = X_train[selected_features]
    X_valid_selected = X_valid[selected_features]

    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train_selected, y_train)

    y_pred_train = model.predict(X_train_selected)
    y_pred_valid = model.predict(X_valid_selected)

    _, pos_is, _, sharpe_is = sharpe_for_vn30f(y_pred_train, y_price[:len(y_pred_train)], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)
    _, pos_os, _, sharpe_oos = sharpe_for_vn30f(y_pred_valid, y_price[len(y_pred_train):], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)

    print('Trade times in sample:',len(pos_is[pos_is != 0]), 'Trade times out sample:',len(pos_os[pos_os != 0]))

    return sharpe_oos, abs((abs(sharpe_is / sharpe_oos))-1)

X_train, X_valid, y_train, y_valid, train_data = split_optuna_data(data)
"""## Define number of trials (no 2)"""

# Create a study object and optimize the objective function
study = optuna.create_study(directions=['maximize', 'minimize'])
unique_trials = 1000

while unique_trials > len(set(str(t.params) for t in study.trials)):
    study.optimize(lambda trial: objective(trial, X_train, X_valid, y_train, y_valid, train_data['Close']), n_trials=1)
    study.trials_dataframe().fillna(0).sort_values('values_0').to_csv('dtree_feature_trials.csv')
    joblib.dump(study, 'dtreemodel.pkl')

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
[I 2024-07-04 12:35:45,354] A new study created in memory with name: no-name-55af94e0-3a32-42ac-bd35-2fd652ff1a7c


A new study created in memory with name: no-name-55af94e0-3a32-42ac-bd35-2fd652ff1a7c


[I 2024-07-04 12:35:46,891] Trial 0 finished with values: [0.9187224747206147, 18.75924089824794] and parameters: {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1}. 


Trade times in sample: 69703 Trade times out sample: 69703
Trial 0 finished with values: [0.9187224747206147, 18.75924089824794] and parameters: {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1}. 


[I 2024-07-04 12:35:50,725] Trial 1 finished with values: [-0.2591376698071776, 69.05256592045005] and parameters: {0: 0, 1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0}. 


...

Trial 1283 failed with parameters: {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0} because of the following error: The value nan is not acceptable.


[W 2024-07-04 13:30:40,819] Trial 1283 failed with value nan.


Trial 1283 failed with value nan.


[I 2024-07-04 13:30:43,308] Trial 1284 finished with values: [0.9424032603346792, 18.324989477558635] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 0, 13: 0, 14: 0}. 


Trade times in sample: 69703 Trade times out sample: 69703
Trial 1284 finished with values: [0.9424032603346792, 18.324989477558635] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 0, 13: 0, 14: 0}. 


In [2]:
study.trials_dataframe().fillna(0).sort_values('values_0')

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_0,params_1,params_2,params_3,...,params_7,params_8,params_9,params_10,params_11,params_12,params_13,params_14,system_attrs_nsga2:generation,state
615,615,-2.797208,5.489778,2024-07-04 13:01:49.785079,2024-07-04 13:01:53.071959,0 days 00:00:03.286880,1,1,1,1,...,0,1,0,1,1,0,1,0,10,COMPLETE
421,421,-2.636168,5.886229,2024-07-04 12:53:38.284134,2024-07-04 12:53:41.347051,0 days 00:00:03.062917,0,1,1,1,...,0,0,1,1,0,1,1,1,7,COMPLETE
1095,1095,-2.421979,6.495217,2024-07-04 13:23:00.022372,2024-07-04 13:23:01.526516,0 days 00:00:01.504144,1,0,0,1,...,0,1,0,0,0,1,0,0,18,COMPLETE
265,265,-2.295823,6.907082,2024-07-04 12:46:32.278420,2024-07-04 12:46:34.937472,0 days 00:00:02.659052,0,0,0,1,...,1,0,1,1,1,1,0,1,4,COMPLETE
491,491,-2.251330,7.063350,2024-07-04 12:56:37.953333,2024-07-04 12:56:41.200170,0 days 00:00:03.246837,0,1,1,1,...,0,1,0,1,1,0,1,0,8,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,157,2.436576,6.450315,2024-07-04 12:42:05.416182,2024-07-04 12:42:07.228181,0 days 00:00:01.811999,0,0,0,0,...,1,1,0,1,1,0,1,0,2,COMPLETE
1057,1057,2.468696,6.405981,2024-07-04 13:21:16.159067,2024-07-04 13:21:17.515400,0 days 00:00:01.356333,0,0,1,0,...,0,1,0,0,0,0,0,0,17,COMPLETE
891,891,2.595786,5.993356,2024-07-04 13:13:58.033848,2024-07-04 13:14:00.478035,0 days 00:00:02.444187,1,0,1,1,...,0,0,1,1,0,1,1,0,15,COMPLETE
87,87,2.615311,5.941147,2024-07-04 12:39:21.303050,2024-07-04 12:39:24.875744,0 days 00:00:03.572694,1,1,1,1,...,1,0,1,1,1,0,1,1,1,COMPLETE
